# Git для Data Science: контроль версий данных с помощью Python и DVC

Данная публикация представляет собой частичный перевод Кристиана Иванчича [Data Version Control With Python and DVC](https://realpython.com/python-data-version-control/).

Машинное обучение и наука о данных сопряжены с рядом задач, отличающихся от традиционной разработки программного обеспечения. [Системы управления версиями](https://ru.wikipedia.org/wiki/%D0%A1%D0%B8%D1%81%D1%82%D0%B5%D0%BC%D0%B0_%D1%83%D0%BF%D1%80%D0%B0%D0%B2%D0%BB%D0%B5%D0%BD%D0%B8%D1%8F_%D0%B2%D0%B5%D1%80%D1%81%D0%B8%D1%8F%D0%BC%D0%B8) помогают разработчикам контролировать изменения исходного кода. Но управление версиями данных, изменения в моделях и датасетах, отлажено не так хорошо.

Если датасет изменился, точно воспроизвести эксперименты, проведенные вами или другими людьми, становится сложной задачей. Многие команды активно разрабатывают инструменты и фреймворки для решения этих проблем.

В этом руководстве мы узнаем как:
- использовать инструмент под названием DVC для решения некоторых из этих проблем;
- корректно отслеживать и редактировать наборы данных и модели;
- совместно использовать одну систему с товарищами по команде;
- делать воспроизводимые эксперименты по машинному обучению.

# Что такое система управления версиями данных

В стандартной разработке программного обеспечения многим людям необходимо работать с общей кодовой базой и обрабатывать несколько версий одного и того же кода. Это может быстро привести к путанице и дорогостоящим ошибкам. Чтобы решить эту проблему, разработчики используют системы управления версиями, такую как Git.

В системе управления версиями есть центральный репозиторий кода, который представляет текущее официальное состояние проекта. Разработчик может сделать копию этого проекта, внести в них изменения и запросить, чтобы новая версия кода стала официальной. В результате запроса код проверяется и тестируется, пока не будет выпущен.

Такие циклы могут происходить по многу раз в день в традиционных проектах, связанных с разработкой. Но в мире промышленного Data Science подобные соглашения и стандарты пока не закрепились. Наличие систем, которые позволяют людям работать быстро и продолжать с того места, на котором остановились другие, повысило бы скорость и качество получаемых результатов. Это позволит людям прозрачно управлять данными, эффективно проводить эксперименты и сотрудничать с другими людьми и командами.

Один из инструментов, помогающих исследователям управлять наборами данных и моделями и проводить воспроизводимые эксперименты — **DVC** (сокр. от Data Version Control).

# Что такое DVC

DVC – это инструмент командной строки, написанный на Python. Он имитирует команды и рабочие процессы Git, чтобы пользователи знакомые с Git, могли легче пользоваться этим инструментом. DVC предназначен для работы вместе с Git. Фактически, команды git и dvc часто используются вместе, одна за другой. В то время как Git используется для хранения версий кода, DVC делает то же самое для файлов моделей и датасетов.

Git может хранить код как локально, так и на хостинге GitHub, Bitbucket или GitLab. Аналогичным образом, DVC позволяет загружать данные на удаленный репозиторий для хранения данных и моделей. Вы можете создать локальную копию удаленного репозитория, изменить файлы, а затем загрузить свои изменения, чтобы поделиться ими с командой.

Удаленный репозиторий может находиться на том же компьютере, на котором вы работаете, или в облаке. DVC поддерживает большинство главных облачных провайдеров, включая AWS, GCP и Azure. Но вы можете настроить удаленный репозиторий DVC на любом другом сервере. С соблюдением мер предосторожности, которые не позволят другим участникам повредить или удалить данные.

При созранении данных в удаленном репозитории, создается `.dvc`-файл. Файл `.dvc` - это небольшой текстовый файл, который описывает текущее состояние файлов данных в удаленном хранилище. За счет небольшого размер `.dvc`-файл может храниться вместе с вашим программным кодом, например, на GitHub.

Далее мы на практике изучим наиболее важные функции, проработав несколько примеров. Перед тем как начать, нам нужно настроить среду для работы, а затем получить некоторые данные.

# Настраиваем рабочую среду

В этом руководстве мы рассмотрим, как использовать DVC, попрактиковавшись на примерах работы с данными изображений. Мы поиграем с большим количеством файлов изображений и обучим модель машинного обучения, которая распознает, что содержит изображение.

Для работы с примерами в системе должны быть установлены [Python 3](https://realpython.com/installing-python/) и [Git](https://git-scm.com/book/en/v2/Getting-Started-Installing-Git).

Поскольку DVC – это инструмент командной строки, необходимо уметь работать в командной строке вашей операционной системы. Если вы пользователь Windows, ознакомьтесь с разделом [«Запуск DVC в Windows» (англ.)](https://dvc.org/doc/user-guide/running-dvc-on-windows).

Чтобы подготовить рабочую среду, нам нужно сделать следующие шаги:
1. Создать и активировать виртуальное окружение.
2. Установить DVC и необходимые библиотеки Python.
3. Создайть форк и клонировать GitHub-репозиторий со всем кодом.
4. Загрузить набор данных для использования в примерах.

Вы можете использовать любой менеджер пакетов и окружений, какой захотите. В этом руководстве мы используем `conda`, поскольку он отлично поддерживает инструменты для анализа данных и машинного обучения. Чтобы создать и активировать виртуальную среду, откройте интерфейс командной строки и введите следующую команду:

```bash
conda create --name dvc python=3.8.2 -y
```

Команда `create` создает новую виртуальную среду. Флаг `--name` дает имя среде – в данном случае `dvc`. Аргумент `python` позволяет выбрать версию Python, которую мы хотим установить в среде. Наконец, флаг `-y` автоматически соглашается установить все необходимые пакеты, которые нужны Python, без необходимости отвечать на какие-либо запросы.

Как только все установлено, активируем среду:

```bash
conda activate dvc
```